# DevPal: Coding Assistant

## Initialize Model

In [26]:
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure()
gemini = genai.GenerativeModel(
    "gemini-2.0-flash",
    generation_config=genai.GenerationConfig(temperature=0.5),
)

## Build the Widgets

In [27]:
from ipywidgets import Text, Dropdown, Textarea, Layout, Button

box_layout = Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    background_color='#2E2E2E',
    padding='20px',
    width='1300px',
    height='800px'
)

init_prompt_input = Text(
    value='',
    placeholder='eg. Generate two random numbers',
    description='Code Description:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='80%', padding='15px', border='solid 2px #444', margin='10px', justify_content='center')
)

feedback_input = Text(
    value='',
    placeholder='eg. Improve code efficiency',
    description='Feedback:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=Layout(width='80%', padding='15px', border='solid 2px #444', margin='10px', justify_content='center')
)

history_display = Textarea(
    value='',
    placeholder='Code History',
    disabled=True,
    layout=Layout(width='40%', height='600px', margin='10px')
)

code_display = Textarea(
    value='',
    placeholder='Current Version',
    disabled=True,
    layout=Layout(width='60%', height='600px', margin='10px')
)

version_dropdown = Dropdown(
    options=tuple(),
    disabled=False,
    layout=Layout(width='40%', height='40px', margin='10px')
)

button = Button(
    description='Send',
    tooltip='Click to Send',
    disabled=False,
    button_style='primary',
    icon='check',
    layout=Layout(width='60%', height='40px', margin='10px', justify_content='center')
)

## Implement Widget Functionalities

In [28]:
from typing import List, Dict, Any
from typing_extensions import LiteralString
from IPython.display import display

chat = gemini.start_chat(history=[])
history: List[str] = []

def submit_prompt(clicked_button: Button) -> None:
    init_prompt, feedback = init_prompt_input.value, feedback_input.value
    _clear_inputs()

    if init_prompt:
        if len(history) > 0:
            _clear_history()

        response = _send_init_prompt(init_prompt)
        _add_to_history(init_prompt, response)
        code_display.value = response

    elif feedback and len(history) > 0:
        improved_response = _send_feedback(feedback)
        _add_to_history(feedback, improved_response)
        code_display.value = improved_response

    _update_history_display()
    _update_dropdown()

def _clear_inputs() -> None:
    init_prompt_input.value = ""
    feedback_input.value = ""

def _clear_history() -> None:
    chat.history.clear()
    history.clear()
    version_dropdown.options = tuple()
    history_display.value = ""

def _send_init_prompt(user_input: str) -> LiteralString:
    prompt = f"Answer to the following prompt with only the code in Python: {user_input}"
    response = chat.send_message(prompt)
    content = _remove_md(response.text)
    return content

def _add_to_history(prompt: str, response: LiteralString) -> None:
    history.insert(0, f"[{len(history) + 1}] {prompt}\n\nResponse:\n{response}")

def _send_feedback(user_input: str) -> LiteralString:
    prompt = f"Adjust the previous code following this instruction: {user_input}"
    response = chat.send_message(prompt)
    content = _remove_md(response.text)
    return content

def _remove_md(text: LiteralString) -> LiteralString:
    return text[len("```python\n") : -len("```")]

def _update_history_display() -> None:
    history_display.value = "\n\n\n".join(history)

def _update_dropdown() -> None:
    qt_versions = len(history)
    version_dropdown.options = tuple(f"Version {index}" for index in range(qt_versions, 1-1, -1))

def on_version_select(change: Dict[str, Any]) -> None:
    if not change['new']:
        return

    chosen_version = int(change['new'].split(" ")[-1])
    qt_versions = len(history)

    while qt_versions > chosen_version:
        for _ in range(2):
            chat.history.pop()
        history.pop(0)
        qt_versions-=1
    
    _update_dropdown()
    code_display.value = _remove_md(chat.history[-1].parts[0].text)
    _update_history_display()

## Add Functionalities to the Widgets

In [29]:
version_dropdown.observe(on_version_select, names='value')
button.on_click(submit_prompt)

## Show Widgets

In [30]:
from ipywidgets import VBox, HBox

layout = VBox([
    HBox([init_prompt_input, feedback_input]),
    HBox([button,version_dropdown]),
    HBox([code_display, history_display])
])

display(layout)